In [11]:
import numpy as np
from scipy.linalg import expm
no_states = 7

In [12]:
# Defining a 7-state model, choosing the rate constants in a convenient way
k_ij = np.full((no_states,no_states),0.3)
k_ij[1][5]=0.5
k_ij[4][2]=0.7
np.fill_diagonal(k_ij, 0)
print(k_ij)

[[0.  0.3 0.3 0.3 0.3 0.3 0.3]
 [0.3 0.  0.3 0.3 0.3 0.5 0.3]
 [0.3 0.3 0.  0.3 0.3 0.3 0.3]
 [0.3 0.3 0.3 0.  0.3 0.3 0.3]
 [0.3 0.3 0.7 0.3 0.  0.3 0.3]
 [0.3 0.3 0.3 0.3 0.3 0.  0.3]
 [0.3 0.3 0.3 0.3 0.3 0.3 0. ]]


In [13]:
# Dummy matrix linear algebra equations
a = np.zeros((no_states, no_states)) #rate of going from state i to j, ii - sum of rates transitioning from i to all other states. (A*x = b)
b = np.zeros(no_states)

for i in range(no_states):
    for j in range(no_states):
        if i != j:
            a[i, j] = k_ij[i, j]
            a[i, i] -= k_ij[i, j]
b[0] = 1 
print(a)
print(b)

[[-1.8  0.3  0.3  0.3  0.3  0.3  0.3]
 [ 0.3 -2.   0.3  0.3  0.3  0.5  0.3]
 [ 0.3  0.3 -1.8  0.3  0.3  0.3  0.3]
 [ 0.3  0.3  0.3 -1.8  0.3  0.3  0.3]
 [ 0.3  0.3  0.7  0.3 -2.2  0.3  0.3]
 [ 0.3  0.3  0.3  0.3  0.3 -1.8  0.3]
 [ 0.3  0.3  0.3  0.3  0.3  0.3 -1.8]]
[1. 0. 0. 0. 0. 0. 0.]


In [14]:
#solving the lin alg equations to find pe
equilibrium_populations = np.linalg.solve(a,b)
equilibrium_populations /= np.sum(equilibrium_populations)
print(equilibrium_populations)

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]


In [15]:
#Choosing an initial population vector
p_init = np.array([1,0,0,0,0,0,0])
#Choice of 21 kappas
no_kappas = (no_states*(no_states-1))//2
kappas= np.full(no_kappas,0.01) 

In [16]:

def listkap_matkappa(kappa):
    matkappa = np.zeros((no_states,no_states))
    triangle = np.triu(np.ones((no_states, no_states), dtype=bool), k=1) 
    matkappa[triangle]= kappa
    matkappa += matkappa.T
    return matkappa

def matkappa_matr(kappa):
    matkappa = listkap_matkappa(kappa)
    matr = np.zeros((no_states,no_states)) 
    matr = matkappa * equilibrium_populations[:, np.newaxis] 
    rdiag = -np.sum(matr,0)
    matr = matr + np.diag(rdiag)
    return matr

In [17]:
matr = matkappa_matr(kappas)
t_diff = 5000
eq_pop_calc = expm(matr*t_diff).dot(p_init)
print(eq_pop_calc)

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
